## Ruby Association Activity Report
# RedAmber - A simple dataframe library for Ruby

### 鈴木弘一 (@heronshoes)


## 今日のお話

- この発表のサポートページがあります。
  - https://github.com/heronshoes/red_amber_activity_report

- [RedAmber](https://github.com/red-data-tools/red_amber) というデータフレームライブラリを作っています
  - Apache Arrow の列指向のデータを扱えるライブラリ
  - Ruby らしい書き方でデータフレームの操作ができることを目指しています
    - 以前はライブラリの紹介文は「A simple dataframe library for Ruby」でしたが、最近「A dataframe library for Rubyists」に変えました

- Grant では主要な機能の実装を完成し、高速化、ドキュメント整備を頑張りました
  - 成果は[最終報告書](https://gist.github.com/heronshoes/d301bfd58adfbf51e28584bf2e68c190) にまとめてあります
  - RubyKaigi と その follow-up eventでもRedAmberの紹介をさせて頂きました（ありがとうございます）
- **今日は Grant 以前と以降の結果も交えてデモを行いながら、RubyらしいデータフレームとしてのRedAmber を作る際に考えていたことをお話しします**

## self.introduction

- 鈴木弘一 (Hirokazu SUZUKI)
- GitHub/ex-Twitter: @heronshoes
- 広島県福山市に住んでいます

  ![福山市](https://www.city.fukuyama.hiroshima.jp/img/common/logo.png) [Batman](https://github.com/heronshoes/red_amber_activity_report/assets/63298319/d94af4c7-2575-4a50-9f00-d28d55e085e7)
- Ruby愛好家というタイプのRubyist
- Red Data Tools / Rubyでデータ処理
- コーヒーとクラフトビールとMINIが好き 今は短歌がマイブームです

よろしくお願いします。

## RedAmber を動かす


In [ ]:
require 'red_amber'
require 'datasets-arrow'  # red-datasets（有名なデータセットを集めたライブラリ）を arrowと連携できるようにしたもの

{ RedAmber: RedAmber::VERSION, Arrow: Arrow::VERSION, Datasets: Datasets::VERSION, Ruby: RUBY_VERSION }

## データフレームとは何か

- データフレームは、列にラベルを持った二次元のデータ構造
- Pythonのpandas、Rのdplyr/tidyr、RustのPolars、RubyでもDaruがある
  - Ruby Association Grantでも過去にデータフレームの取り組みがあった
- SQLのテーブルと共通する部分が多い概念

![DataFrame in RedAmber](https://github.com/heronshoes/red_amber_activity_report/assets/63298319/4379918e-86ff-4bdd-b6a2-69aacbcaa365)

![dataframe model of RedAmber](https://raw.githubusercontent.com/red-data-tools/red_amber/main/doc/image/dataframe_model.png)

```python
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['col_0', 'col_1', 'col_2', 'col_3'],
                  index=['row_0', 'row_1', 'row_2'])
print(df)
#        col_0  col_1  col_2  col_3
# row_0      0      1      2      3
# row_1      4      5      6      7
# row_2      8      9     10     11
```

RedAmberでは、行インデックスは特別扱いしないで、一つの列として扱う


In [ ]:
require 'arrow-numo-narray'
n_rows, n_cols = 3, 4
RedAmber::DataFrame.new(index: (0...n_rows).map { "row_#{_1}" })
  .assign do
    narray = Numo::Int8.new(n_rows, n_cols).seq
    (0...n_cols).map { |i| ["col_#{i}", narray[true, i]] }
  end

![Vectorのメソッド](https://github.com/heronshoes/red_amber_activity_report/assets/63298319/e9416899-5219-4551-a059-a836dd633900)

参考）[Apache ArrowとRedAmber](https://github.com/heronshoes/red_amber_activity_report/assets/63298319/22852887-2eed-4330-95bf-249c2a39c80b)

## Penguins データを読み込んでみる

元データは、Rの　palmerpenguins データセット,
Red Datasetsから取得できる.


In [ ]:
penguins = RedAmber::DataFrame.new(Datasets::Penguins.new)

データフレームの要約を行と列を転置して表示させる `#tdr` メソッド
- type : データの型 (Arrow Array の型)
- level : ユニークな要素の数 (カテゴリー数)
- データは tallyした形式または部分 Array で表示、nil と NaN の数も表示する


In [ ]:
penguins.tdr

Rでは,
```
str(penguins)

#> tibble [344 × 8] (S3: tbl_df/tbl/data.frame)
#>  $ species          : Factor w/ 3 levels "Adelie","Chinstrap",..: 1 1 1 1 1 1 1 1 1 1 ...
#>  $ island           : Factor w/ 3 levels "Biscoe","Dream",..: 3 3 3 3 3 3 3 3 3 3 ...
#>  $ bill_length_mm   : num [1:344] 39.1 39.5 40.3 NA 36.7 39.3 38.9 39.2 34.1 42 ...
#>  $ bill_depth_mm    : num [1:344] 18.7 17.4 18 NA 19.3 20.6 17.8 19.6 18.1 20.2 ...
#>  $ flipper_length_mm: int [1:344] 181 186 195 NA 193 190 181 195 193 190 ...
#>  $ body_mass_g      : int [1:344] 3750 3800 3250 NA 3450 3650 3625 4675 3475 4250 ...
#>  $ sex              : Factor w/ 2 levels "female","male": 2 1 1 NA 1 2 1 2 NA NA ...
#>  $ year             : int [1:344] 2007 2007 2007 2007 2007 2007 2007 2007 2007 2007 ...
```

`#tdr` は、transposed dataframe representation の頭文字

※ (RedAmberで工夫したことその1) **データフレームの内容をコンパクトに理解するための `#tdr` メソッドを作った**

数値的な要約＝統計量を表示する
- count : Varid な要素の数 (nil, NaNを除く）
- pandas とは違って、転置して表示している


In [ ]:
penguins.summary

In [ ]:
penguins.summary.transpose

## Diamonds データセットを読み込んで例を示す

元データは、Rの　「Prices of over 50,000 round cut diamonds」,
Red Datasetsから利用できる.


In [ ]:
diamonds = RedAmber::DataFrame.new(Datasets::Diamonds.new)

In [ ]:
diamonds.tdr

In [ ]:
diamonds.carat

In [ ]:
diamonds.carat.mean

In [ ]:
diamonds.carat.quantiles

In [ ]:
diamonds.carat > 1
# (diamonds.carat > 1).tally

In [ ]:
diamonds.slice(diamonds.carat > 1)
# diamonds.slice { diamonds.carat > 1 }
# filter

In [ ]:
diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  # .pick { vectors.map { _1.numeric? } }
  # .pick { keys.select { _1.to_s.start_with?('c') } }

In [ ]:
diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  .group(:cut)

In [ ]:
diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  .group(:cut)
  .mean

In [ ]:
diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  .group(:cut)
  .mean
  .sort('-mean(price)')

In [ ]:
diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  .group(:cut)
  .mean
  .sort('-mean(price)')
  .rename('mean(price)': :mean_price_usd)

In [ ]:
usdjpy = 110.0  # when the yen was stronger

diamonds
  .slice { carat > 1 }
  .pick(:cut, :price)
  .group(:cut)
  .mean
  .sort('-mean(price)')
  .rename('mean(price)': :mean_price_usd)
  # 新しい列を作る assign { [列のラベル, 列の値ベクトル] }
  .assign { [:mean_price_jpy, mean_price_usd * usdjpy] }

  # 新しい列を作る assign(列のラベル) { 列の値ベクトル }
  # .assign(:mean_price_jpy) { mean_price_usd * usdjpy } 

## データフレームの基本操作

- 列を選択する: `#pick`, 列を取り除く: `#drop`
- 行を選択する: `#slice`, 行を取り除く: `#remove`
  - `[行 ： 列]`でアドレッシングするやり方は、列指向でベクトル演算を主体とする場合は使われにくい
  - その代わり、選択指定子はインデックスの配列でも、ブーリアン配列でもどちらでも受け付けるようにした


In [ ]:
diamonds.slice { carat > 1 }
# puts diamonds.inspect
# diamonds.slice { [0..10, -10..-1] }

### DataFrames 操作メソッドの比較

RedAmberの基本的な操作メソッドを [Python pandas](https://pandas.pydata.org/),
[R Tidyverse](https://www.tidyverse.org/),
[Julia DataFrames](https://dataframes.juliadata.org/stable/) と比較

#### 列 (variables) を選択する

| 機能                               | RedAmber        | Tidyverse (R)	                  | pandas                                 | DataFrames.jl     |
|---                                 |---              |---                              |---                                     |---                |
| 列を選択して dataframe で返す       | **pick**, drop, []  | dplyr::select, dplyr::select_if | [], loc[], iloc[], drop, select_dtypes | [], select        |
| 列を選択して vector で返す          | [], v	         | dplyr::pull, [, x]	           | [], loc[], iloc[]                      | [!, :x]           |
| 列の順番を入れ替えた dataframeを返す | **pick**, [] 	     | relocate                        | [], reindex, loc[], iloc[]             | select,transform  |

列の中身をベクトルとして取り出す


In [ ]:
diamonds.carat
# diamonds[:carat]
# diamonds.v(:carat)

In [ ]:
diamonds1 = diamonds.rename(carat: 'carat-value')
# diamonds1.carat-value
diamonds1['carat-value']
# diamonds1.v('carat-value')

In [ ]:
diamonds1.slice { v('carat-value') > 1 }
# diamonds1.slice { self['carat-value'] > 1 }

#### 行 (records, observations) を選択する

| 機能                                     | RedAmber 	               | Tidyverse (R)               | pandas                   | DataFrames.jl |
|---                                       |---                        |---                          |---                       |---            |
| 論理値に従って行を選択して dataframe で返す | **slice**, filter, remove, [] | dplyr::filter               | [], filter, query, loc[] | filter        |
| インデックスで行を選択して dataframe で返す | **slice**, remove, []         | dplyr::slice 	            | iloc[], drop             | subset        |
| 行の順番を入れ替えた dataframeを返す       | **slice**, [] 	             | dplyr::filter, dplyr::slice | reindex, loc[], iloc[]   | permute       |

#### 列を更新する / 新しい列を作る

|機能 	                       | RedAmber 	          | Tidyverse (R)                                     | pandas            | DataFrames.jl |
|---                           |---                  |---                                                 |---                |---            |
| 既存の列の内容を変更する       | **assign** 	            | dplyr::mutate                                    	| assign, []=       | mapcols       |
| 新しい列を作成する 	        | **assign**, assign_left |	dplyr::mutate 	                                    | apply             | insertcols,.+ |
| 新しい列を作成し、残りは捨てる | new 	               | transmute 	                                         | (dfply:)transmute | transform,insertcols,mapcols |
| 列の名前を変更する            | **rename**              |	dplyr::rename, dplyr::rename_with, purrr::set_names | rename, set_axis  | rename        |
| dataframe をソートする        | sort 	              | dplyr::arrange 	                                    | sort_values       | sort          |


In [ ]:
diamonds
  .pick(:depth, :z)
  .assign(:depth, :'diff(depth-z)') do
    depth_mm = depth / 25.4
    diff = depth_mm - z
    [depth_mm, diff]
  end

![RedAmberの基本的なメソッド群](https://raw.githubusercontent.com/heronshoes/red_amber/main/doc/image/basic_verbs.png)

※ (RedAmberで工夫したことその2) **Rubyのブロックと親和性が高い、整理されたメソッド群を作った**

### よくハマってしまうミス

ブロックを `instance_eval` を使ってレシーバのコンテキストでコンテキストで呼んでいる。そして`method_missing`で列名をメソッドとして呼べるようにしている。このため、

- 列名とメソッド名が同じ
- 列名とローカル変数が同じ

どちらも予期しないメソッドが呼ばれて、ミスに気づきにくい


In [ ]:
diamonds
# diamonds.table

In [ ]:
diamonds.x
diamonds.slice { x > 5 }

In [ ]:
x = 0
diamonds.slice { x > 5 }

Ruby は LISP-2 だから変数名とメソッド名は別の空間に存在できるけれども、適切に管理するべき。

## データフレームの横持ちと縦持ち

メーカー別、年別の輸入車台数データ。日本輸入車組合（ https://www.jaia-jp.org/ja/stats/ ）より。


In [ ]:
# csvからロードする
import_cars = RedAmber::DataFrame.load(Arrow::Buffer.new(<<~CSV), format: :csv)
  Year,Audi,BMW,BMW_MINI,Mercedes-Benz,VW
  2017,28336,52527,25427,68221,49040
  2018,26473,50982,25984,67554,51961
  2019,24222,46814,23813,66553,46794
  2020,22304,35712,20196,57041,36576
  2021,22535,35905,18211,51722,35215
CSV

このデータフレームは「Messy」なデータと呼ばれ、データ分析や可視化に適していないとされています。

日本語では、表が横に長いことから「横持ち」のデータと呼ばれます。

このデータをデータ分析や可視化に適した「Tidy　（縦持ち）」なデータフレームにするためには、`#to_long`を使います。

（Tidy, Messyは　Rにおけるデータ処理の神様である　Hadely Wickham の用語です)


In [ ]:
import_cars.to_long

# (:Year, name: :Manufactures, value: :n_of_imported)

縦持ちのデータを横持ちに戻すには`#to_wide`を使う。


In [ ]:
df_wide = df_long.to_wide

# (name: :Year, value: :n_of_imported)

In [ ]:
df_wide.transpose

![データフレームの変形(reshape)](https://raw.githubusercontent.com/heronshoes/red_amber/main/doc/image/reshaping_dataframe.png)

### dataframe を変形するメソッドの比較

| 機能 	                                | RedAmber  | Tidyverse (R)       | pandas      | DataFrames.jl |
|---                                   |---        |---                  |---           |---            |
| 列を行に積む (long dataframe にする)   | to_long   | tidyr::pivot_longer | melt         | stack         |
| 行を列に集める (wide dataframe にする) | to_wide   | tidyr::pivot_wider  | pivot        | unstack       |
| wide dataframe を転置する             | transpose | transpose, t 	      | transpose, T | permutedims   |

※ (RedAmberで工夫したことその3) **良い感じに操作できるデータフレームの変形操作**

## Groupの取扱い

グループはデータフレームでよく登場する重要な概念。例えば売り上げのデータがあったとして、

- 店舗毎の売り上げ合計を求める
- 月毎に売り上げ合計上位３位にある商品を求める

などのタスクの「毎」を取り扱うのがグループ！

※ (RedAmberで工夫したことその3) **忘れていても思い出しながら使える、データフレームの変形操作を作った**

## グループ化とその周辺

グループ化とは、いくつかの列の値に基づいてデータフレームを**グループに分け**、それぞれの値を**集約する**操作。

関連する操作に、分析関数（ウィンドウ）関数とローリングウィンドウ関数がある。分析関数とは、（私の理解では）集約しないグループ化であり、ローリングウィンドウ関数の代表的な使い方は移動平均である。

![グループ化](https://user-images.githubusercontent.com/63298319/221889155-21a37b7c-ef41-4f41-a72a-a2d4da983591.png)


In [ ]:
dataframe = RedAmber::DataFrame.new(
  x: 0..5,
  y: %w[A A B B B C],
  z: [false, true, false, nil, true, false]
)

In [ ]:
dataframe.group(:y)

In [ ]:
dataframe.group(:y).sum
# dataframe.group(:y).summarize { sum(:x) }a

In [ ]:
dataframe.x

![SubFramesコンセプト](https://user-images.githubusercontent.com/63298319/213947955-9e4d7a5f-0315-4a0e-bd09-484137645fca.png)

Rのtidyrでは下記のように「nest」という概念を導入している。

これはデータフレームの内部にデータフレームを取り込む概念といえる。RedAmberのSubFramesは、データフレームの集合セットを外部に広げていくような考え方だと思っている。

![参考:Rのデータフレームにおけるnest](https://user-images.githubusercontent.com/63298319/224402474-0d02d8ca-64fc-450f-a44e-c65c727d1226.png)

SubFramesの場合：

![SubFrames](https://user-images.githubusercontent.com/63298319/224340732-4558b2a4-61dd-458d-a3bf-85e230a4a3dc.png)


In [ ]:
dataframe
  .sub_by_value(:y)

In [ ]:
dataframe
  .sub_by_value(:y)
  .aggregate do
    {
      y: y.one,
      sum_x: x.sum,
    }
  end

![移動平均](https://user-images.githubusercontent.com/63298319/224350718-020def87-7894-418f-83d1-80ca99f60f07.png)


In [ ]:
dataframe
  .sub_by_window(size: 3)
  .aggregate do
    {
      mean_x: x.mean,
      count_z: z.count,
    }
  end

集約しないグループをSubFramesでやる

![assign](https://user-images.githubusercontent.com/63298319/224435740-ebf6af40-3aa4-4855-8c20-e957e3a321f2.png)


In [ ]:
dataframe
  .assign do
    {
      x: indices(1),
      c: x.cumsum,
    }
  end

In [ ]:
dataframe
  .sub_by_value(:y)
  .assign do
    {
      x: indices(1),
      c: x.cumsum,
    }
  end
  # .concatenate

データフレームの部分集合としてのSubFramesオブジェクトにより、サブデータフレームの処理は単体のデータフレームの処理と全く同じスタイルで書ける。

実際、`SubFrames#assign` の定義は `DataFrame`に丸投げである。
```ruby
  class SubFrames
    def assign(...)
      map { |df| df.assign(...) }
    e
  endnd
```

一方で、効率が損なわれていることがSubFramesの欠点。

## その他、Grantで頑張ったこと

- 初期の実装に比べて大幅に高速化できた
- ドキュメント整備を頑張った（YARDドキュメントカバー率100%、exampleも豊富に）
- テストカバー率を頑張った（100%）

## まだまだこれからなこと

- Arrowの改良に合わせて機能を拡充していく
- SubFramesが遅い
- QueryをLazyに実行する仕組み
- Rubyデータ処理全般を盛り上げていく

## 総括
- Rubyで整備が遅れていたデータフレームの分野で、Arrowデータを直接取り扱えるライブラリを開発することができた。
- 「Rubyらしく書けるデータフレーム処理」を、ブロックによる記述、Ruby汎用のコレクションクラスの多用などを通じて実現した。
- データフレームの主要な機能をほぼカバーできた。
- 操作例の豊富なドキュメントを整備した。
- 10月4日(v0.2.2)..3月11日(v0.4.1)のコードの差は、77ファイルが変更され、14702行の追加、2637行の削除であった。（ドキュメントの寄与が大きい）
- コードを見直し、高速化を図った。
- テストカバー率を100%に引上げ、一定のコード品質を確保した。
- 他のデータフレームライブラリにない、新しい「SubFrames」という概念を導入し実験的機能として実装した。シンプルかつRubyらしい一貫した書き方でグループやウィンドウの処理ができることを示した。速度の向上は今後の課題である。
- 上流のRed Arrowに対して必要な機能を充実させるための提案を行うとともに、テーブル結合操作の改善にも貢献できた。
- RedAmberの開発のヒントになったライブラリRoverの作者Andrew Kane氏は、RustのPolarsをベースにしたPolars-rubyを誕生させた。Polarsは高速性に特徴があるArrow Formatを読み書きできるライブラリで今後が非常に期待できる。RedAmberにもLazyFrameなどの先進的な機能を取り入れていきたい。

## 謝辞
 このプロジェクトの実施にあたっては、メンターの村田(@mrkn)さんに常に的確な助言と温かいサポートを賜りました。 須藤(@kou)さんにはRed ArrowのコミットやRedAmberの不具合に対する助言と共に、Red Data Toolsの月例配信でもお世話になりました。 Benson Muite(@bkmgit)さんにはFedraのテストワークフローの追加と、他のデータフレームとの比較表のJuliaの部分を追加して頂きました。 @kojix2さんにはYARDドキュメント生成ワークフローの追加、ドキュメントの修正などでコードに貢献して頂きました。またそれ以外でもメソッド名に対する議論などでRed Data ToolsのGitterでメンバーの方々には貴重なご意見やご提案を頂いております。皆様の支援に対しましてこの場をお借りして深く感謝申し上げます。
 
最後にこのような機会を与えてくださったRuby Assciation、Rubyを生みそして育ててくださっているまつもとさんはじめRubyコミュニティの皆様に深く感謝申し上げます。
